In [219]:
import requests
import re
from sqlalchemy import create_engine
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import mysql.connector as mysql

# Scraping Data from web

In [222]:
url = 'https://www.myjobmag.co.ke/'
req = requests.get(url).content
soup = bs(req,'html.parser')

In [223]:
names = soup.find_all('li',class_= "mag-b")
name = []
for i in names:
    a = i.find('a')
    name.append(a.text)
name

['Vacancies at Kenya Methodist University (KeMU)',
 'Vacancies at U.S. Embassy to Kenya',
 'Job Vacancies at Mwananchi Credit Ltd',
 'Latest Recruitment at Aga Khan University Hospital',
 'Vacancies at Nova Pioneer',
 'Vacancies at Plan International',
 'Job Opportunities at Compassion International',
 'Job Openings at Canonical',
 'Retail Operations Coordinator at Human Asset Consultants Ltd',
 'Executive Sous Chef at Mambo Italia',
 'Business Development Manager – Graduate Programs at Strathmore Business School',
 'ICT Technical Support Officer at CIC Insurance',
 'Temporary Financial Accountant at The Pharo Foundation',
 'Logistics Specialist at SoCha',
 'Production Assistant Manager at Grounded',
 'Human Resource & Culture Lead at Safaricom Investment Co-operative (S.I.C.)',
 'Trade Marketing & Distribution - Office Internship at British American Tobacco (BAT)',
 'Human Resources Change Management Specialist at Kenyans.co.ke',
 'Vacancies at Pam Nail Care Ltd',
 'Private Label Asso

In [224]:
descs = soup.find_all('li',class_='job-desc')
desc = []
for i in descs:
    desc.append(i.text.strip())
desc

['Kenya Methodist University (KeMU) is a Chartered Private Christian Institution of higher learning which aims to contribute to the transformation of our society by providing high quality education that...',
 'The U.S. Embassy to Kenya opened in 1964. Through sixteen Ambassadors since then, the Embassy has enriched the ties between the American and Kenyan people and served the needs of Americans in and arou...',
 'Mwananchi Credit Limited was incorporated as a private limited company under the Companies Act (Cap 486) of the Laws of Kenya. It initially started operations under the names Mwananchi Microlink Ltd which was incorporated on 15th April 2010. On 10th January 2012, it later changed its names to Mwananchi Credit Limited.',
 'Aga Khan University Hospitals in Karachi, Pakistan and Nairobi, Kenya are private, not-for-profit institutions providing high quality health care. The Main Hospitals serve as the principal sites for c...',
 'Nova Pioneer Education Group is a Pan-African inde

In [225]:
dates = soup.find_all('li',id='job-date')
datelist = []
for i in dates:
    adyear = i.text.strip() + ' 2024'
    convert = datetime.strptime(adyear , "%d %B %Y")
    format = convert.strftime("%d-%m-%Y")
    datelist.append(format)
datelist

['29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024',
 '29-08-2024']

In [226]:
h2 = soup.find_all('li' ,class_='mag-b')

link = []
for i in h2:
    a = i.find('a', href=True)
    convert = a.get('href')
    format = url + convert
    link.append(format)
link

['https://www.myjobmag.co.ke//jobs/vacancies-at-kenya-methodist-university-kemu-2',
 'https://www.myjobmag.co.ke//jobs/vacancies-at-u-s-embassy-to-kenya-2',
 'https://www.myjobmag.co.ke//jobs/job-vacancies-at-mwananchi-credit-ltd',
 'https://www.myjobmag.co.ke//jobs/latest-recruitment-at-aga-khan-university-hospital-7',
 'https://www.myjobmag.co.ke//jobs/administrative-assistant-tatu-girls-school-at-nova-pioneer',
 'https://www.myjobmag.co.ke//jobs/vacancies-at-plan-international-11',
 'https://www.myjobmag.co.ke//jobs/trips-and-visits-specialist-i-at-compassion-international',
 'https://www.myjobmag.co.ke//jobs/accounts-payable-clerk-entry-level-opportunity-at-canonical-1',
 'https://www.myjobmag.co.ke//job/retail-operations-coordinator-human-asset-consultants-ltd',
 'https://www.myjobmag.co.ke//job/executive-sous-chef-mambo-italia',
 'https://www.myjobmag.co.ke//job/business-development-manager-graduate-programs-strathmore-business-school',
 'https://www.myjobmag.co.ke//job/ict-techn

In [227]:
data = {
    'Title':name,
    'Desc': desc,
    'Date': datelist,
    'Job Link':link
}
job_df = pd.DataFrame(data)
# f = job_df.transpose()
job_df.head()

,Title,Desc,Date,Job Link
0,Vacancies at Kenya Methodist University (KeMU),Kenya Methodist University (KeMU) is a Charter...,29-08-2024,https://www.myjobmag.co.ke//jobs/vacancies-at-...
1,Vacancies at U.S. Embassy to Kenya,The U.S. Embassy to Kenya opened in 1964. Thro...,29-08-2024,https://www.myjobmag.co.ke//jobs/vacancies-at-...
2,Job Vacancies at Mwananchi Credit Ltd,Mwananchi Credit Limited was incorporated as a...,29-08-2024,https://www.myjobmag.co.ke//jobs/job-vacancies...
3,Latest Recruitment at Aga Khan University Hosp...,"Aga Khan University Hospitals in Karachi, Paki...",29-08-2024,https://www.myjobmag.co.ke//jobs/latest-recrui...
4,Vacancies at Nova Pioneer,Nova Pioneer Education Group is a Pan-African ...,29-08-2024,https://www.myjobmag.co.ke//jobs/administrativ...


# Connect & Load to MySQL

In [243]:
connect = mysql.connect(host = '127.0.0.1',
user = 'root',
password = '')
cursor = connect.cursor()

In [244]:
cursor.execute('use simple_etl;')

In [237]:
host = '127.0.0.1'
user = 'root'
password = ''
dbname = 'simple_etl'
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{dbname}')
job_df.to_sql('webdate', con=engine , index=False , if_exists='replace' )
engine.dispose()
connect.close()